In [1]:
import numpy as np
import ee
import folium
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt
import seaborn as sns

import keras
from keras import layers
from keras import models

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X_train = np.load("/content/drive/My Drive/TFM/train-test-val/X_train.npy")
y_train = np.load("/content/drive/My Drive/TFM/train-test-val/y_train.npy")

X_val = np.load("/content/drive/My Drive/TFM/train-test-val/X_val.npy")
y_val = np.load("/content/drive/My Drive/TFM/train-test-val/y_val.npy")

In [0]:
X_test_AL = np.load("/content/drive/My Drive/TFM/train-test-val/X_test_AL.npy")
y_test_AL = np.load("/content/drive/My Drive/TFM/train-test-val/y_test_AL.npy")

X_test_Bubal = np.load("/content/drive/My Drive/TFM/train-test-val/X_test_Bubal.npy")
y_test_Bubal = np.load("/content/drive/My Drive/TFM/train-test-val/y_test_Bubal.npy")

X_test_Canelles = np.load("/content/drive/My Drive/TFM/train-test-val/X_test_Canelles.npy")
y_test_Canelles = np.load("/content/drive/My Drive/TFM/train-test-val/y_test_Canelles.npy")

X_test_Grado = np.load("/content/drive/My Drive/TFM/train-test-val/X_test_Grado.npy")
y_test_Grado = np.load("/content/drive/My Drive/TFM/train-test-val/y_test_Grado.npy")

In [0]:
scaler = StandardScaler().fit(X_train[:,2:8])
X_train_scaled = scaler.transform(X_train[:,2:8])
X_val_scaled = scaler.transform(X_val[:,2:8])

In [0]:
X_test_scaled_AL = scaler.transform(X_test_AL[:,2:8])
X_test_scaled_Bubal = scaler.transform(X_test_Bubal[:,2:8])
X_test_scaled_Canelles = scaler.transform(X_test_Canelles[:,2:8])
X_test_scaled_Grado = scaler.transform(X_test_Grado[:,2:8])

In [43]:
model = models.Sequential()

model.add(layers.Dense(2**3, activation='sigmoid', input_shape = (X_train_scaled[:,2:8].shape[1],)))
model.add(layers.Dense(2**3, activation='sigmoid'))
model.add(layers.Dense(1))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8)                 32        
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras import optimizers

#model.compile(optimizer=optimizers.RMSprop(lr = 1e-4),loss="mse")
model.compile(optimizer=optimizers.Adadelta(),loss="mse")

In [0]:
from keras import callbacks

callback_es = callbacks.EarlyStopping(patience=10)

In [46]:
history = model.fit(X_train_scaled[:,2:8],y_train,epochs = 50,callbacks=[callback_es],validation_data=(X_val_scaled[:,2:8],y_val))

Train on 335548 samples, validate on 18644 samples
Epoch 1/50
335548/335548 [==============================] - 12s 35us/step - loss: 364.8893 - val_loss: 257.1835
Epoch 2/50
335548/335548 [==============================] - 12s 37us/step - loss: 252.9076 - val_loss: 254.3379
Epoch 3/50
335548/335548 [==============================] - 12s 35us/step - loss: 251.6004 - val_loss: 253.4513
Epoch 4/50
335548/335548 [==============================] - 12s 36us/step - loss: 250.4255 - val_loss: 252.3662
Epoch 5/50
335548/335548 [==============================] - 12s 35us/step - loss: 248.8908 - val_loss: 250.4404
Epoch 6/50
335548/335548 [==============================] - 12s 35us/step - loss: 245.8715 - val_loss: 244.2884
Epoch 7/50
335548/335548 [==============================] - 12s 35us/step - loss: 238.4175 - val_loss: 240.2028
Epoch 8/50
335548/335548 [==============================] - 12s 35us/step - loss: 236.1652 - val_loss: 238.7387
Epoch 9/50
335548/335548 [===========================

In [47]:
y_pred_AL = model.predict(X_test_scaled_AL[:,2:8])
y_pred_Bubal = model.predict(X_test_scaled_Bubal[:,2:8])
y_pred_Canelles = model.predict(X_test_scaled_Canelles[:,2:8])
y_pred_Grado = model.predict(X_test_scaled_Grado[:,2:8])

from sklearn.metrics import mean_squared_error as mse

print("AL:",mse(y_test_AL,y_pred_AL,squared = False))
print("Bubal:",mse(y_test_Bubal,y_pred_Bubal,squared = False))
print("Canelles:",mse(y_test_Canelles,y_pred_Canelles,squared = False))
print("Grado:",mse(y_test_Grado,y_pred_Grado,squared = False))

AL: 9.107234377486286
Bubal: 5.622745472134463
Canelles: 18.154732038837594
Grado: 12.800261129804522


In [33]:
y_pred_AL = model.predict(X_test_scaled_AL[:,2:8])
y_pred_Bubal = model.predict(X_test_scaled_Bubal[:,2:8])
y_pred_Canelles = model.predict(X_test_scaled_Canelles[:,2:8])
y_pred_Grado = model.predict(X_test_scaled_Grado[:,2:8])

from sklearn.metrics import mean_squared_error as mse

print("AL:",mse(y_test_AL,y_pred_AL,squared = False))
print("Bubal:",mse(y_test_Bubal,y_pred_Bubal,squared = False))
print("Canelles:",mse(y_test_Canelles,y_pred_Canelles,squared = False))
print("Grado:",mse(y_test_Grado,y_pred_Grado,squared = False))

AL: 9.055884152529542
Bubal: 5.977847901695689
Canelles: 18.176764169426733
Grado: 12.599270382871513
